In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer,VectorAssembler
spark = SparkSession.builder.appName('nlp').getOrCreate()
data = spark.read.options(inferSchema=True,heeader=True).csv('/FileStore/tables/SMSSpamCollection',sep='\t')
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data = data.withColumn('length', length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [0]:
data.groupBy(data['class']).mean('length').show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham| 71.4545266210897|
| spam|138.6706827309237|
+-----+-----------------+



In [0]:
# tokenizer, stop word remover, count vectorizer, idf, string index
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remover = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
class_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [0]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [0]:
nb = NaiveBayes()

In [0]:
data_prep_pipeline = Pipeline(stages=[class_to_numeric,tokenizer,stop_remover,count_vec,idf,clean_up])

In [0]:
cleaner = data_prep_pipeline.fit(data)
clean_data = cleaner.transform(data)

In [0]:
clean_data = clean_data.select('label','features')
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [0]:
train,test = clean_data.randomSplit([0.7,0.3])
spam_dectector = nb.fit(train)

In [0]:
test_results = spam_dectector.transform(test)

In [0]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1064.3090614818...|[1.0,4.3743215596...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-997.01744194888...|[1.0,1.7782716517...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-536.39082782814...|[1.0,8.3232611654...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-991.10482332584...|[1.0,6.8727108963...|       0.0|
|  0.0|(13424,[0,1,27,85...|[-1552.7762767192...|[0.00109534578488...|       1.0|
|  0.0|(13424,[0,1,43,69...|[-615.32309202620...|[0.99990543918068...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1140.2774206356...|[2.01790729181148...|       1.0|
|  0.0|(13424,[0,1,881,1...|[-96.986569276508...|[0.99999999134141...|       0.0|
|  0.0|(13424,[0,1,4305,...|[-144.19686013095...|[0.00426934202688...|       1.0|
|  0.0|(13424,[0

In [0]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
acc

Out[46]: 0.9295715768583985